In [46]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

warnings.simplefilter(action='ignore', category=Warning)

from xgboost import XGBClassifier
# !pip install lightgbm
from lightgbm import LGBMClassifier
# !pip install catboost
# from catboost import CatBoostClassifier

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: '%.3f' % x)
pd.set_option("display.width", None)


from funcs.data_prep import *
from funcs.eda import *

In [8]:
df = pd.read_csv("HousePrice/train.csv")

In [9]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
df.shape

(1460, 81)

In [47]:
check_df(df)

##################### Shape #####################
(1460, 81)
##################### Types #####################
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2  

In [36]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 1460
Variables: 81
cat_cols: 53
num_cols: 27
cat_but_car: 1
num_but_cat: 11


In [17]:
for col in cat_cols:
    cat_summary(df, col)

         MSZoning  Ratio
RL           1151 78.836
RM            218 14.932
FV             65  4.452
RH             16  1.096
C (all)        10  0.685
#######################
      Street  Ratio
Pave    1454 99.589
Grvl       6  0.411
#######################
      Alley  Ratio
Grvl     50  3.425
Pave     41  2.808
#######################
     LotShape  Ratio
Reg       925 63.356
IR1       484 33.151
IR2        41  2.808
IR3        10  0.685
#######################
     LandContour  Ratio
Lvl         1311 89.795
Bnk           63  4.315
HLS           50  3.425
Low           36  2.466
#######################
        Utilities  Ratio
AllPub       1459 99.932
NoSeWa          1  0.068
#######################
         LotConfig  Ratio
Inside        1052 72.055
Corner         263 18.014
CulDSac         94  6.438
FR2             47  3.219
FR3              4  0.274
#######################
     LandSlope  Ratio
Gtl       1382 94.658
Mod         65  4.452
Sev         13  0.890
#####################

In [21]:
rare_analyser(df, "SalePrice", cat_cols)

MSZoning : 5
         COUNT  RATIO  TARGET_MEAN
C (all)     10  0.007    74528.000
FV          65  0.045   214014.062
RH          16  0.011   131558.375
RL        1151  0.788   191004.995
RM         218  0.149   126316.830


Street : 2
      COUNT  RATIO  TARGET_MEAN
Grvl      6  0.004   130190.500
Pave   1454  0.996   181130.539


Alley : 2
       COUNT  RATIO  TARGET_MEAN
Alley                           
Grvl      50  0.034   122219.080
Pave      41  0.028   168000.585


LotShape : 4
     COUNT  RATIO  TARGET_MEAN
IR1    484  0.332   206101.665
IR2     41  0.028   239833.366
IR3     10  0.007   216036.500
Reg    925  0.634   164754.818


LandContour : 4
     COUNT  RATIO  TARGET_MEAN
Bnk     63  0.043   143104.079
HLS     50  0.034   231533.940
Low     36  0.025   203661.111
Lvl   1311  0.898   180183.747


Utilities : 2
           COUNT  RATIO  TARGET_MEAN
Utilities                           
AllPub      1459  0.999   180950.957
NoSeWa         1  0.001   137500.000


LotConfig : 5
 

In [22]:
for col in num_cols:
    num_summary(df, col)

count   1460.000
mean     730.500
std      421.610
min        1.000
5%        73.950
10%      146.900
20%      292.800
30%      438.700
40%      584.600
50%      730.500
60%      876.400
70%     1022.300
80%     1168.200
90%     1314.100
95%     1387.050
99%     1445.410
max     1460.000
Name: Id, dtype: float64
count   1460.000
mean      56.897
std       42.301
min       20.000
5%        20.000
10%       20.000
20%       20.000
30%       20.000
40%       30.000
50%       50.000
60%       60.000
70%       60.000
80%       80.000
90%      120.000
95%      160.000
99%      190.000
max      190.000
Name: MSSubClass, dtype: float64
count   1201.000
mean      70.050
std       24.285
min       21.000
5%        34.000
10%       44.000
20%       53.000
30%       60.000
40%       63.000
50%       69.000
60%       74.000
70%       79.000
80%       85.000
90%       96.000
95%      107.000
99%      141.000
max      313.000
Name: LotFrontage, dtype: float64
count     1460.000
mean     10516.828
std

In [24]:
df.shape

(1460, 81)

In [38]:
for col in num_cols:
    print(col, check_outlier(df, col))

Id False
MSSubClass True
LotFrontage True
LotArea True
OverallQual True
YearBuilt True
YearRemodAdd False
MasVnrArea True
BsmtFinSF1 True
BsmtFinSF2 True
BsmtUnfSF True
TotalBsmtSF True
1stFlrSF True
2ndFlrSF True
LowQualFinSF True
GrLivArea True
TotRmsAbvGrd True
GarageYrBlt False
GarageArea True
WoodDeckSF True
OpenPorchSF True
EnclosedPorch True
3SsnPorch True
ScreenPorch True
MiscVal True
MoSold False
SalePrice True


In [43]:
outlier_index = []
for col in num_cols:
    reach_outliers(df, col, q1=0.25, q3=0.75, index=True)

Empty DataFrame
Columns: [Id, MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, MiscVal, MoSold, YrSold, SaleType, SaleCondition, SalePrice]
Index: []
    Id  MSSubClass MSZoning 

[Int64Index([], dtype='int64'),
 Int64Index([   9,   48,   56,   75,   87,   93,  115,  125,  145,  165,
             ...
             1304, 1334, 1358, 1364, 1367, 1378, 1393, 1416, 1449, 1452],
            dtype='int64', length=103),
 Int64Index([  36,   41,   56,   75,   85,   86,  145,  150,  159,  171,  193,
              195,  197,  225,  227,  231,  232,  235,  260,  276,  277,  290,
              302,  306,  313,  363,  429,  430,  432,  434,  446,  489,  500,
              504,  523,  599,  614,  649,  655,  662,  675,  758,  807,  825,
              829,  837,  889,  906,  909,  915,  934,  956,  959,  962,  963,
              966,  991, 1007, 1010, 1029, 1038, 1039, 1061, 1088, 1091, 1104,
             1106, 1107, 1127, 1151, 1160, 1168, 1169, 1173, 1182, 1191, 1211,
             1219, 1273, 1291, 1298, 1334, 1337, 1339, 1359, 1361, 1378, 1449],
            dtype='int64'),
 Int64Index([  53,   66,  113,  120,  159,  171,  185,  197,  249,  260,  271,
              277,  304,